In [227]:
import os

In [228]:
%pwd

'c:\\'

In [229]:
os.chdir("../")

In [230]:
%pwd

'c:\\'

In [231]:
import pandas as pd


In [232]:
data = pd.read_csv(r"C:\Users\DELL\Documents\End-to-end-Machine-learning-project\artifacts\data_ingestion\winequality-red.csv")
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5
1,7.8,0.88,0.00,2.6,0.098,25.0,67.0,0.9968,3.20,0.68,9.8,5
2,7.8,0.76,0.04,2.3,0.092,15.0,54.0,0.9970,3.26,0.65,9.8,5
3,11.2,0.28,0.56,1.9,0.075,17.0,60.0,0.9980,3.16,0.58,9.8,6
4,7.4,0.70,0.00,1.9,0.076,11.0,34.0,0.9978,3.51,0.56,9.4,5


In [233]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [234]:
data.isnull().sum()

fixed acidity           0
volatile acidity        0
citric acid             0
residual sugar          0
chlorides               0
free sulfur dioxide     0
total sulfur dioxide    0
density                 0
pH                      0
sulphates               0
alcohol                 0
quality                 0
dtype: int64

In [235]:
data.shape

(1599, 12)

In [236]:
data.columns

Index(['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar',
       'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density',
       'pH', 'sulphates', 'alcohol', 'quality'],
      dtype='object')

In [237]:
from dataclasses import dataclass
from pathlib import Path
import pandas as pd

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

In [238]:
from mlProject.constants import *  # Make sure this import works
from mlProject.utils.common import read_yaml, create_directories

In [239]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config.data_validation
        schema = self.schema.COLUMNS

        create_directories([config.root_dir])

        data_validation_config = DataValidationConfig(
            root_dir=config.root_dir,
            STATUS_FILE=config.STATUS_FILE,
            unzip_data_dir = config.unzip_data_dir,
            all_schema=schema,
        )

        return data_validation_config



In [240]:
import os
from mlProject import logger 

In [241]:
class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config


    def validate_all_columns(self)-> bool:
        try:
            validation_status = True

            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = list(data.columns)
            print(self.config.all_schema)
            print(self.config.STATUS_FILE)

            all_schema = self.config.all_schema.keys()

            
            for col in all_cols:
                if col not in all_schema:
                    validation_status = False
                    with open(self.config.STATUS_FILE, 'w') as f:
                        f.write(f"Validation status: {validation_status}")
                    break
              
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status
        
        except Exception as e:
            raise e 


In [242]:
try:
    config = ConfigurationManager()
    data_validation_config = config.get_data_validation_config()
    data_validation = DataValidation(config=data_validation_config)
    print(data_validation.validate_all_columns())
except Exception as e:
    raise e

[2024-10-14 23:38:04,515: INFO: common: yaml file: C:\Users\DELL\Documents\End-to-end-Machine-learning-project\config\config.yaml loaded successfully]
[2024-10-14 23:38:04,517: INFO: common: yaml file: C:\Users\DELL\Documents\End-to-end-Machine-learning-project\params.yaml loaded successfully]
[2024-10-14 23:38:04,517: INFO: common: yaml file: C:\Users\DELL\Documents\End-to-end-Machine-learning-project\schema.yaml loaded successfully]
[2024-10-14 23:38:04,517: INFO: common: created directory at: artifacts]
[2024-10-14 23:38:04,517: INFO: common: created directory at: artifacts/data_validation]
{'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sulfur dioxide': 'float64', 'density': 'float64', 'pH': 'float64', 'sulphates': 'float64', 'alcohol': 'float64', 'quality': 'int64'}
C:\Users\DELL\Documents\End-to-end-Machine-learning-project\research\artifacts\data_va

In [243]:
import pandas as pd
from pathlib import Path
from dataclasses import dataclass
import os
import logging

# Assuming logger is set up
logger = logging.getLogger(__name__)

@dataclass(frozen=True)
class DataValidationConfig:
    root_dir: Path
    STATUS_FILE: str
    unzip_data_dir: Path
    all_schema: dict

class ConfigurationManager:
    def __init__(self, config_filepath, params_filepath, schema_filepath):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        # Create directories
        create_directories([self.config['artifacts_root']])

    def get_data_validation_config(self) -> DataValidationConfig:
        config = self.config['data_validation']
        create_directories([config['root_dir']])

        return DataValidationConfig(
            root_dir=Path(config['root_dir']).resolve(),
            STATUS_FILE=config['STATUS_FILE'],
            unzip_data_dir=Path(config['unzip_data_dir']).resolve(),
            all_schema=self.schema
        )

class DataValidation:
    def __init__(self, config: DataValidationConfig):
        self.config = config

    def validate_all_columns(self) -> bool:
        try:
            validation_status = True

            # Load the CSV data for validation
            data = pd.read_csv(self.config.unzip_data_dir)
            all_cols = [col.strip() for col in data.columns]  # Strip whitespace from column names

            # Print the actual columns present in the data
            print("Actual Data Columns:", all_cols)
            print(self.config.all_schema)

            # Get expected schema columns
            all_schema_columns = self.config.all_schema.COLUMNS.keys()
            
            print("Expected Schema Columns:", list(all_schema_columns))
            

            # Identify missing columns
            missing_columns = [col for col in all_schema_columns if col not in all_cols]
            if missing_columns:
                validation_status = False
                print(f"Missing columns in data: {missing_columns}")  # Print missing columns
            else:
                print("All expected columns are present.")

            # Write the validation status to the status file
            with open(self.config.STATUS_FILE, 'w') as f:
                f.write(f"Validation status: {validation_status}")

            return validation_status

        except Exception as e:
            logger.exception("Error occurred during validation")
            raise e

# Usage
if __name__ == "__main__":
    try:
        config = ConfigurationManager(
            config_filepath=Path(r"C:\Users\DELL\Documents\End-to-end-Machine-learning-project\config\config.yaml").resolve(),
            params_filepath=Path(r"C:\Users\DELL\Documents\End-to-end-Machine-learning-project\params.yaml").resolve(),
            schema_filepath=Path(r"C:\Users\DELL\Documents\End-to-end-Machine-learning-project\schema.yaml").resolve()
        )
        data_validation_config = config.get_data_validation_config()
        data_validation = DataValidation(config=data_validation_config)

        # Perform the column validation
        validation_status = data_validation.validate_all_columns()
        print(f"Validation Status: {validation_status}")  # Print the result of the validation

    except Exception as e:
        logger.exception("An error occurred during the validation process.")


[2024-10-14 23:38:04,580: INFO: common: yaml file: C:\Users\DELL\Documents\End-to-end-Machine-learning-project\config\config.yaml loaded successfully]
[2024-10-14 23:38:04,582: INFO: common: yaml file: C:\Users\DELL\Documents\End-to-end-Machine-learning-project\params.yaml loaded successfully]
[2024-10-14 23:38:04,585: INFO: common: yaml file: C:\Users\DELL\Documents\End-to-end-Machine-learning-project\schema.yaml loaded successfully]
[2024-10-14 23:38:04,587: INFO: common: created directory at: artifacts]
[2024-10-14 23:38:04,589: INFO: common: created directory at: artifacts/data_validation]
Actual Data Columns: ['fixed acidity', 'volatile acidity', 'citric acid', 'residual sugar', 'chlorides', 'free sulfur dioxide', 'total sulfur dioxide', 'density', 'pH', 'sulphates', 'alcohol', 'quality']
{'COLUMNS': {'fixed acidity': 'float64', 'volatile acidity': 'float64', 'citric acid': 'float64', 'residual sugar': 'float64', 'chlorides': 'float64', 'free sulfur dioxide': 'float64', 'total sul